In [30]:
import pandas as pd
import pandas_ta as ta
import talib as TA
from backtesting import Strategy, Backtest
from datetime import datetime
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

In [ ]:
'''
This strategy is based on support and resistance levels mixed with candlestick patterns.
Initially, the strategy will use 4H data of EURUSD, but I'll add more timeframes later.
The backtest is done with backtesting.py and is intended to throw the results from it.
'''

LOAD AND CLEAN DATA

In [47]:
df = pd.read_csv('../data_sets/EURUSD_Candlestick_4_Hour_BID_01.03.2014-02.04.2022.csv')
df = df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df['Local time'] = df['Local time'].str.rstrip('GMT-0500')
df['Date'] = pd.to_datetime(df['Local time'])
df.drop(columns=['Local time'], inplace=True)
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
df['Date1'] = df['Date']
df.set_index('Date', inplace=True)
df.tail()

,Open,High,Low,Close,Volume,Date1
Date,,,,,,
2022-03-31 20:00:00,1.10673,1.10730,1.10613,1.10637,21587.74,2022-03-31 20:00:00
2022-01-04 00:00:00,1.10638,1.10757,1.10428,1.10605,55565.31,2022-01-04 00:00:00
2022-01-04 04:00:00,1.10605,1.10677,1.10365,1.10438,53602.25,2022-01-04 04:00:00
2022-01-04 08:00:00,1.10438,1.10536,1.10279,1.10402,28882.14,2022-01-04 08:00:00
2022-01-04 12:00:00,1.10402,1.10524,1.10325,1.10427,15225.47,2022-01-04 12:00:00


CALCULATION OF ATR FOR SL & TP

In [6]:
df['ATR']= df.ta.atr()
df.tail()

,Open,High,Low,Close,Volume,ATR
Date,,,,,,
2022-03-31 20:00:00,1.10673,1.10730,1.10613,1.10637,21587.74,0.003656
2022-01-04 00:00:00,1.10638,1.10757,1.10428,1.10605,55565.31,0.003630
2022-01-04 04:00:00,1.10605,1.10677,1.10365,1.10438,53602.25,0.003593
2022-01-04 08:00:00,1.10438,1.10536,1.10279,1.10402,28882.14,0.003520
2022-01-04 12:00:00,1.10402,1.10524,1.10325,1.10427,15225.47,0.003411


CALCULATING SUPPORT & RESISTANCE

In [8]:
def support(df, l, n1, n2):
    # checking if the lows before "l" are decreasing
    for i in range(l-n1+1, l+1):
        if df.Low[i] > df.Low[i-1]:
            return 0
    # checking if the lows after "l" are increasing    
    for i in range(l+1, l+n2+1):
        if df.Low[i] < df.Low[i-1]:
            return 0
    
    return 1
        

def resistance(df, l, n1, n2):
    # checking if the highs before "l" are decreasing
    for i in range(l-n1+1, l+1):
        if df.High[i] < df.High[i-1]:
            return 0
    # checking if the highs after "l" are increasing    
    for i in range(l+1, l+n2+1):
        if df.High[i] > df.High[i-1]:
            return 0
    
    return 1

CHECK FOR SUPPORT & RESISTANCE LEVELS

In [ ]:
sr = []
n1 = 2
n2 = 2

for row in range(2, 12605):
    if support(df, row, n1, n2):
        sr.append((row, df.Low[row],1))
    if resistance(df, row, n1, n2):
        sr.append((row, df.High[row],2))
        

FILTER FOR VERY CLOSE LINES

In [ ]:
plotlist1 = [x[1] for x in sr if x[2] == 1]
plotlist2 = [x[1] for x in sr if x[2] == 2]
plotlist1.sort()
plotlist2.sort()

for i in range(1, len(plotlist1)):
    if(i>=len(plotlist1)):
        break
    if abs(plotlist1[i] - plotlist1[i-1]) <= 0.005:
        plotlist1.pop(i)
        
for i in range(1, len(plotlist2)):
    if(i>=len(plotlist2)):
        break
    if abs(plotlist2[i] - plotlist2[i-1]) <= 0.005:
        plotlist2.pop(i)
plotlist1

BACKTEST USING ATR FOR SL & TP

In [11]:
class SRCP(Strategy):
    
    def init(self):
        super().init()
        self.engulfing = self.I(TA.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close, penetration=0)
        self.sstar = self.I(TA.CDLSHOOTINGSTAR, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.hammer = self.I(TA.CDLHAMMER, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.sr = self.I(sr)
    
    def next(self):
        super().next()
        pass